In [1]:
from transformers import pipeline
from transformers import BertForMaskedLM
from transformers import BertTokenizer
import torch.nn
import torch
from torch.nn import functional as F
from transformers import RepetitionPenaltyLogitsProcessor
import torch.nn as nn

In [2]:
# RUN THIS TO GET ~6HR TRAINED V2 MODEL WITH W2V EMBEDDINGS
MODEL = './polished/models/v2bert/bert_model/'
TOKENIZER = './polished/models/v2bert/berttokenizer/'

In [3]:
# RUN THIS TO GET ~2HR TRAINED V2 MODEL WITHOUT W2V EMBEDDINGS
MODEL = './polished/models/ka_only_no_w2v_bert/ka_only_no_w2v_bert_model//'
TOKENIZER = './polished/models/v2bert/berttokenizer/' # tokenizer same

In [4]:
# BEST MODEL
# RUN THIS TO GET 10HR TRAINED V1 MODEL
MODEL = './polished/models/bert/model/'
TOKENIZER = './polished/models/bert/berttokenizer/'

In [5]:
model = BertForMaskedLM.from_pretrained(MODEL)
tokenizer = BertTokenizer.from_pretrained (TOKENIZER)

Some weights of the model checkpoint at ./polished/models/bert/model/ were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
repetition_penalty=True

In [7]:
def infer_next_token_softmax(token_ids: list[int]): # we are inferring next token. last token should not be [SEP]
    assert token_ids[-1] != tokenizer.sep_token_id
    i = len(token_ids)
    token_ids = token_ids + [tokenizer.mask_token_id] # + [tokenizer.pad_token_id]*(50-len(token_ids)-10) + [tokenizer.sep_token_id] + [tokenizer.sep_token_id]
    token_ids = torch.tensor(token_ids).view((1, -1))
    #attention_mask = (token_ids != tokenizer.pad_token_id)*1
    #print(attention_mask)
    next_token_scores = model(input_ids=token_ids).logits[0, i, :] 
    logits = F.softmax(next_token_scores, dim=0)
    #print(logits.shape)
    if repetition_penalty:
        logits = RepetitionPenaltyLogitsProcessor(10.0)(token_ids.view(1, -1), logits.view(1, -1))
    return logits.view(-1)

In [8]:
def get_next_top_k(token_ids: list[int], k: int):
    probs = infer_next_token_softmax(token_ids)
    tops = list(reversed(sorted([(float(v), i) for i, v in enumerate(probs)])))[:k]
    return [(p, i) for p, i in tops] # p, token_id

In [22]:
def beam_search(sentence: str, num_tokens: int, k: int = 5):
    assert num_tokens >= 1
    token_ids = tokenizer(sentence)['input_ids'][:-1]
    cur = [(1.0, [])]
    for _ in range(num_tokens):
        nexts = []
        for p, i in cur:
            nexts += [(c_p*p, i+[j]) for c_p, j in get_next_top_k(token_ids + i, k)]
        cur = list(reversed(sorted(nexts)))[:k]
    return ([tokenizer.decode(token_ids[1:]+toks) for _, toks in cur])

In [23]:
beam_search('1+1=', 1, 5)

['1 + 1 = 2', '1 + 1 = /', '1 + 1 = 3', '1 + 1 = 4', '1 + 1 = 1']

In [24]:
beam_search('1+1=', 3, 5)

['1 + 1 = 2 - 3',
 '1 + 1 = 2, 4',
 '1 + 1 = 2, 3',
 '1 + 1 = 3, 2',
 '1 + 1 = 2 - 4']

In [25]:
beam_search('პრეზიდენტი მიხეილ', 1, 5)

['პრეზიდენტი მიხეილ ივანიშვილის',
 'პრეზიდენტი მიხეილ ბიძინა',
 'პრეზიდენტი მიხეილ ზურაბიშვილი',
 'პრეზიდენტი მიხეილ სააკაშვილის',
 'პრეზიდენტი მიხეილ სააკაშვილი']

In [26]:
beam_search('პრეზიდენტი მიხეილ სააკ', 1, 5)

['პრეზიდენტი მიხეილ სააკაშვილი',
 'პრეზიდენტი მიხეილ სააკაშვილის',
 'პრეზიდენტი მიხეილ სააკაძე',
 'პრეზიდენტი მიხეილ სააკაძის',
 'პრეზიდენტი მიხეილ სააკიძის']

In [27]:
beam_search('პრეზიდენტი მიხეილ', 5, 5)

['პრეზიდენტი მიხეილ ზურაბიშვილი : გიორგი ვაშაძე,',
 'პრეზიდენტი მიხეილ სააკაშვილი, რომ განაცხადა -',
 'პრეზიდენტი მიხეილ სააკაშვილი, რომ აღნიშნა -',
 'პრეზიდენტი მიხეილ სააკაშვილი, რომ ამბობს,',
 'პრეზიდენტი მიხეილ ზურაბიშვილი : გიორგი ვაშაძე -']

In [31]:
beam_search('დღეს მე ვერტმფრენით გავფრინდი', num_tokens = 10)

['დღეს მე ვერტმფრენით გავფრინდი და აი, შენ არ ვიცი, რა არის ეს',
 'დღეს მე ვერტმფრენით გავფრინდი და აი, შენ არ ვიცი, რომ იმიტომ რომ',
 'დღეს მე ვერტმფრენით გავფრინდი და აი, შენ არ ვიცი, რომ იმიტომ,',
 'დღეს მე ვერტმფრენით გავფრინდი და აი, შენ არ ვიცი რა უნდა რომ ასე',
 'დღეს მე ვერტმფრენით გავფრინდი და აი, შენ არ ვიცი, რომ ასე ვარ']

In [30]:
beam_search('პრეზიდენტი მიხეილ', num_tokens = 10)

['პრეზიდენტი მიხეილ ზურაბიშვილი : გიორგი ვაშაძე, პრემიერ - მინისტრის საგარეო საქმეთა',
 'პრეზიდენტი მიხეილ ზურაბიშვილი : გიორგი ვაშაძე, პრემიერ - მინისტრმა საგარეო საქმეთა',
 'პრეზიდენტი მიხეილ ზურაბიშვილი : გიორგი ვაშაძე, პრემიერ - მინისტრის შინაგან საქმეთა',
 'პრეზიდენტი მიხეილ ზურაბიშვილი : გიორგი ვაშაძე, პრემიერ - მინისტრი საგარეო საქმეთა',
 'პრეზიდენტი მიხეილ ზურაბიშვილი : გიორგი ვაშაძე, პრემიერ - მინისტრის საქმეთა საგარეო']

# Below is MLM demo

### Masked Language Modelling

In [206]:
from transformers import pipeline

In [207]:
fill = pipeline('fill-mask', model=MODEL, tokenizer=TOKENIZER)

Some weights of the model checkpoint at ./polished/models/v2bert/bert_model/ were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [208]:
fill('პრეზიდენტი მიხეილ [MASK] აზრით ამ შენობის აშენება კარგი იდეაა.')

[{'score': 0.15355493128299713,
  'token': 5,
  'token_str': '!',
  'sequence': 'პრეზიდენტი მიხეილ! აზრით ამ შენობის აშენება კარგი იდეაა.'},
 {'score': 0.08936669677495956,
  'token': 720,
  'token_str': '##ის',
  'sequence': 'პრეზიდენტი მიხეილის აზრით ამ შენობის აშენება კარგი იდეაა.'},
 {'score': 0.08673388510942459,
  'token': 9245,
  'token_str': 'საათამდე',
  'sequence': 'პრეზიდენტი მიხეილ საათამდე აზრით ამ შენობის აშენება კარგი იდეაა.'},
 {'score': 0.03182349354028702,
  'token': 4499,
  'token_str': 'გამყიდველისაგან',
  'sequence': 'პრეზიდენტი მიხეილ გამყიდველისაგან აზრით ამ შენობის აშენება კარგი იდეაა.'},
 {'score': 0.028807684779167175,
  'token': 747,
  'token_str': '##აც',
  'sequence': 'პრეზიდენტი მიხეილაც აზრით ამ შენობის აშენება კარგი იდეაა.'}]

In [209]:
fill('საქართველოს საუკეთესო კერძი არის [MASK], ცომში გახვეული ხორცი.')

[{'score': 0.280554860830307,
  'token': 1916,
  'token_str': '##შვილი',
  'sequence': 'საქართველოს საუკეთესო კერძი არისშვილი, ცომში გახვეული ხორცი.'},
 {'score': 0.12002026289701462,
  'token': 4996,
  'token_str': 'გურამ',
  'sequence': 'საქართველოს საუკეთესო კერძი არის გურამ, ცომში გახვეული ხორცი.'},
 {'score': 0.10072647035121918,
  'token': 2435,
  'token_str': '##იძე',
  'sequence': 'საქართველოს საუკეთესო კერძი არისიძე, ცომში გახვეული ხორცი.'},
 {'score': 0.06962481886148453,
  'token': 4238,
  'token_str': 'ილია',
  'sequence': 'საქართველოს საუკეთესო კერძი არის ილია, ცომში გახვეული ხორცი.'},
 {'score': 0.03749911114573479,
  'token': 1683,
  'token_str': 'ხან',
  'sequence': 'საქართველოს საუკეთესო კერძი არის ხან, ცომში გახვეული ხორცი.'}]

In [210]:
fill('საუკეთესო [MASK] კერძი არის ხინკალი, ცომში გახვეული ხორცი.')

[{'score': 0.14893898367881775,
  'token': 1916,
  'token_str': '##შვილი',
  'sequence': 'საუკეთესოშვილი კერძი არის ხინკალი, ცომში გახვეული ხორცი.'},
 {'score': 0.10062244534492493,
  'token': 6648,
  'token_str': '1924',
  'sequence': 'საუკეთესო 1924 კერძი არის ხინკალი, ცომში გახვეული ხორცი.'},
 {'score': 0.053386539220809937,
  'token': 2435,
  'token_str': '##იძე',
  'sequence': 'საუკეთესოიძე კერძი არის ხინკალი, ცომში გახვეული ხორცი.'},
 {'score': 0.048771098256111145,
  'token': 10863,
  'token_str': 'მაზრა',
  'sequence': 'საუკეთესო მაზრა კერძი არის ხინკალი, ცომში გახვეული ხორცი.'},
 {'score': 0.048658207058906555,
  'token': 520,
  'token_str': '##ს',
  'sequence': 'საუკეთესოს კერძი არის ხინკალი, ცომში გახვეული ხორცი.'}]